<a href="https://colab.research.google.com/github/ShivaCharan347/sentiment_text_analysis/blob/main/Movie_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import re
import spacy
import json
import pandas as pd
import fasttext
from collections import Counter
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [5]:
# !pip install fasttext

In [17]:
# !python -m spacy download en_core_web_md

In [6]:
df = pd.read_csv('movie.metadata.tsv', sep='\t', header = None, usecols=[0,2,8])
df

,0,2,8
0,975900,Ghosts of Mars,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,"{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,Brun bitter,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,White Of The Eye,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,A Woman in Flames,"{""/m/07s9rl0"": ""Drama""}"
...,...,...,...
81736,35228177,Mermaids: The Body Found,"{""/m/07s9rl0"": ""Drama""}"
81737,34980460,Knuckle,"{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,Another Nice Mess,"{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,The Super Dimension Fortress Macross II: Lover...,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [7]:
df.rename(columns= {0:'Movie_id', 8:'Genre_type', 2:'Movie_Tilte'}, inplace=True)

In [8]:
for i in range(len(df)):
    jsn_dta = json.loads(df['Genre_type'][i])
    df['Genre_type'][i] = list(jsn_dta.values())

<ipython-input-8-eef3a5eaa002>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genre_type'][i] = list(jsn_dta.values())


In [9]:
cln_df=pd.read_csv("plot_summaries.txt",sep='\t',header=None)
cln_df

,0,1
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...
...,...,...
18487,31320148,"The setting is Stockholm, Sweden, ""this year""...."
18488,28509475,"In La nudité toute nue , director Olivier Nick..."
18489,20530554,The inhabitants of the forest that Woody Woodp...
18490,35019469,Farmon bibi is a wise and loving but strict m...


In [11]:
cln_df.rename(columns={0:'Movie_id', 1:'Summary'}, inplace=True)

In [12]:
mv_df = pd.merge(df, cln_df, on= 'Movie_id')

In [13]:
mv_df.drop(['Movie_Tilte'], axis = 1, inplace = True)

#Cleaning Data
for i in range(len(mv_df)):
    a=re.sub("\W"," ",mv_df["Summary"][i])
    b=re.sub("\s\s+"," ",a)
    b = b.lower()
    mv_df["Summary"][i]=b
mv_df["Summary"]

0        a series of murders of rich young women throug...
1        eva an upper class housewife becomes frustrate...
2         plot dateact 1act 2act 3act 4act 5 finally ne...
3        serap a young actress with a strong lively per...
4         the story starts as one of the robots flies i...
                               ...                        
18443    joe morton stars in this dramatic comedy set i...
18444    at the beginning there is a scene with a dog p...
18445    a story about a young man who inherits a broke...
18446    kalai arasi is a well blended story of romance...
18447    the story takes place in the year 2092 the sup...
Name: Summary, Length: 18448, dtype: object

In [14]:
mv_df

,Movie_id,Genre_type,Summary
0,9363483,"[Thriller, Erotic thriller, Psychological thri...",a series of murders of rich young women throug...
1,261236,[Drama],eva an upper class housewife becomes frustrate...
2,171005,"[Costume drama, War film, Epic, Period piece, ...",plot dateact 1act 2act 3act 4act 5 finally ne...
3,18296435,[Comedy],serap a young actress with a strong lively per...
4,11250635,"[Science Fiction, Adventure, Animation, Short ...",the story starts as one of the robots flies i...
...,...,...,...
18443,1673588,"[Science Fiction, Indie, Cult, Drama, Comedy, ...",joe morton stars in this dramatic comedy set i...
18444,30863646,"[Culture & Society, Horror, Mondo film, World ...",at the beginning there is a scene with a dog p...
18445,20244619,"[Drama, World cinema]",a story about a young man who inherits a broke...
18446,19286279,"[Romance Film, Action, Drama, Science Fiction]",kalai arasi is a well blended story of romance...


In [18]:
nlp = spacy.load('en_core_web_md')

In [19]:
for i in range(len(mv_df)):
    doc=nlp(mv_df["Summary"][i])
    list1=[]
    for token in doc:
        if token.is_stop==False:
            list1.append(str(token.lemma_))
    mv_df["Summary"][i]=" ".join(list1)
mv_df

,Movie_id,Genre_type,Summary
0,9363483,"[Thriller, Erotic thriller, Psychological thri...",series murder rich young woman arizona bear di...
1,261236,[Drama],eva upper class housewife frustrated leave arr...
2,171005,"[Costume drama, War film, Epic, Period piece, ...",plot dateact 1act 2act 3act 4act 5 finally n...
3,18296435,[Comedy],serap young actress strong lively personality ...
4,11250635,"[Science Fiction, Adventure, Animation, Short ...",story start robot fly scientist s secret lai...
...,...,...,...
18443,1673588,"[Science Fiction, Indie, Cult, Drama, Comedy, ...",joe morton star dramatic comedy set new york c...
18444,30863646,"[Culture & Society, Horror, Mondo film, World ...",beginning scene dog pound dog pound dog presum...
18445,20244619,"[Drama, World cinema]",story young man inherit broken estate edge per...
18446,19286279,"[Romance Film, Action, Drama, Science Fiction]",kalai arasi blend story romance science fictio...


In [20]:
Genres = []
for i in range(len(mv_df)):
    Genres.extend(mv_df['Genre_type'][i])

In [21]:
Counter(Genres)

Counter({'Thriller': 2837,
         'Erotic thriller': 78,
         'Psychological thriller': 485,
         'Drama': 8358,
         'Costume drama': 175,
         'War film': 691,
         'Epic': 98,
         'Period piece': 570,
         'Film adaptation': 519,
         'Comedy': 4566,
         'Science Fiction': 1019,
         'Adventure': 1403,
         'Animation': 1085,
         'Short Film': 1400,
         'Fantasy': 874,
         'Family Film': 1422,
         'Action': 2606,
         "Children's/Family": 335,
         'Musical': 1079,
         "Children's Fantasy": 104,
         'Romantic comedy': 870,
         'Romance Film': 2885,
         'World cinema': 2195,
         'Indie': 1537,
         'Fan film': 6,
         'Biographical film': 282,
         'Historical fiction': 139,
         'Black-and-white': 1612,
         'Biopic [feature]': 162,
         'Culture & Society': 97,
         'Biography': 255,
         'Documentary': 539,
         'Christmas movie': 42,
         'T

In [22]:
train_df, test_df = train_test_split(mv_df.drop(['Movie_id'], axis=1), test_size=0.2, random_state=13)

In [23]:
train_data = []

with open('concept_train.txt', 'w', encoding='utf-8') as train_file:

    for i, row in train_df.iterrows():

        labels = ' '.join(f'__label__{genre.replace(" ", "_")}' for genre in row['Genre_type'])

        line = f'{labels} {row["Summary"]}\n'

        train_file.write(line)

        train_data.append(line)

# Step 2: Prepare the testing data file

test_data = []

with open('concept_test.txt', 'w', encoding='utf-8') as test_file:

    for i, row in test_df.iterrows():

        labels = ' '.join(f'__label__{genre.replace(" ", "_")}' for genre in row['Genre_type'])

        line = f'{labels} {row["Summary"]}\n'

        test_file.write(line)

        test_data.append(line)

In [27]:
# Step 3: Train and evaluate the FastText model for different epochs

for i in [20, 25, 30]:

    model_sup = fasttext.train_supervised(input='concept_train.txt', lr=0.5, epoch=i, wordNgrams=3, bucket=20000)

    result = model_sup.test('concept_test.txt', k=1, threshold=0.1)

    test_num, precision, recall = result

    f_score = 2 * precision * recall / (precision + recall)

    print(f"Epoch: {i}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f_score:.2f}")

# Step 4: Print words and Labels from the trained model

for word, label in zip(model_sup.words, model_sup.labels):

    print(word, label)

Epoch: 20, Precision: 0.60, Recall: 0.16, F1 Score: 0.25
Epoch: 25, Precision: 0.60, Recall: 0.16, F1 Score: 0.25
Epoch: 30, Precision: 0.59, Recall: 0.16, F1 Score: 0.25
s __label__Drama
</s> __label__Comedy
find __label__Romance_Film
man __label__Thriller
leave __label__Action
tell __label__World_cinema
kill __label__Crime_Fiction
take __label__Horror
go __label__Black-and-white
try __label__Action/Adventure
film __label__Indie
father __label__Family_Film
love __label__Adventure
friend __label__Short_Film
life __label__Romantic_drama
come __label__Animation
time __label__Musical
return __label__Science_Fiction
day __label__Mystery
new __label__Fantasy
family __label__Romantic_comedy
home __label__Comedy_film
meet __label__Crime_Thriller
end __label__War_film
house __label__Japanese_Movies
help __label__Period_piece
begin __label__Comedy-drama
mother __label__Documentary
year __label__Film_adaptation
work __label__Psychological_thriller
later __label__Bollywood
decide __label__Silent_

In [26]:
# Step 5: Test and evaluate the FastText model for different 'K' & 'threshold' values
import numpy as np

for k in range(2, 6):
    for j in np.arange(0.1, 1.1, 0.1):
        result = model_sup.test('concept_test.txt', k=k, threshold=j)

        test_num, precision, recall = result
        f_score = 2 * precision * recall / (precision + recall)
        print(f"Epoch: 30, k: {k:.1f}, Threshold: {j:.1f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f_score:.2f}")

for word, label in zip(model_sup.words, model_sup.labels):
    print(word, label)

Epoch: 30, k: 2.0, Threshold: 0.1, Precision: 0.54, Recall: 0.24, F1 Score: 0.33
Epoch: 30, k: 2.0, Threshold: 0.2, Precision: 0.62, Recall: 0.14, F1 Score: 0.23
Epoch: 30, k: 2.0, Threshold: 0.3, Precision: 0.65, Recall: 0.08, F1 Score: 0.15
Epoch: 30, k: 2.0, Threshold: 0.4, Precision: 0.68, Recall: 0.06, F1 Score: 0.10
Epoch: 30, k: 2.0, Threshold: 0.5, Precision: 0.71, Recall: 0.04, F1 Score: 0.07
Epoch: 30, k: 2.0, Threshold: 0.6, Precision: 0.70, Recall: 0.02, F1 Score: 0.04
Epoch: 30, k: 2.0, Threshold: 0.7, Precision: 0.65, Recall: 0.01, F1 Score: 0.02
Epoch: 30, k: 2.0, Threshold: 0.8, Precision: 0.65, Recall: 0.01, F1 Score: 0.01
Epoch: 30, k: 2.0, Threshold: 0.9, Precision: 0.67, Recall: 0.00, F1 Score: 0.00
Epoch: 30, k: 2.0, Threshold: 1.0, Precision: nan, Recall: 0.00, F1 Score: nan
Epoch: 30, k: 3.0, Threshold: 0.1, Precision: 0.52, Recall: 0.26, F1 Score: 0.35
Epoch: 30, k: 3.0, Threshold: 0.2, Precision: 0.61, Recall: 0.14, F1 Score: 0.23
Epoch: 30, k: 3.0, Threshold: 

In [28]:
result = model_sup.test('concept_test.txt', k=4, threshold=0.1)
# Epoch: 30, k: 4.0, Threshold: 0.1, Precision: 0.52, Recall: 0.26, F1 Score: 0.35

In [29]:
a = 'This short documentary film tells the story of Alfari who lives on the bank of the Niger, a river that is slowly running dry due to climate change. Alfari had to give up fishing to become a gardener, fighting against the hippopotamus that devastates his plantations'

In [30]:
model_sup.predict(a,k=4) #threshold=0.1
#array --prob

(('__label__Drama',
  '__label__Comedy',
  '__label__Black-and-white',
  '__label__Short_Film'),
 array([0.41288036, 0.12689871, 0.0707359 , 0.0591642 ]))